<a href="https://colab.research.google.com/github/shino11/Tesis_Deteccion-de-ironia-y-casificacion-de-Polaridad-en-rese-as-de-usuarios-de-Trip-Advisor/blob/main/sgd_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montar Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install & Verification

In [ ]:
# !pip install -U tensorflow==2.2
# !pip install -U numpy==1.19.5
# !pip install h5py==2.10.0


#!pip uninstall tensorflow
#!pip install tensorflow
#!pip uninstall numpy
#!pip install numpy
#!apt-get install python3.7

#import sys
#import tensorflow as tf


#tf.__version__
#!python3 ‐‐version
#print(sys.version)

!pip show scikit-learn pandas tensorflow numpy

###Dnn_Algorithm_Part1

In [ ]:
import io
from builtins import print
import numpy as np
#import keras
import tensorflow as tf
#print('la version de tensorflow es:'+tf.__version__)
import pandas as pd
#from keras import Model
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
#from tensorflow.core.protobuf import rewriter_config_pb2
import os
# from keras.engine.saving import model_from_json
# from keras.models import Sequential
# from keras.layers import Dense, Flatten, Lambda
# from keras.layers import LSTM, Input
# from keras.layers import Dense, Embedding, Reshape
# from keras.layers.wrappers import TimeDistributed
# from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
#from keras.utils.np_utils import to_categorical
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, balanced_accuracy_score, classification_report
#from statsmodels.sandbox.distributions.examples.ex_mvelliptical import fig
from tensorflow.keras import Model
from tensorflow.python.keras.layers.core import Dropout, RepeatVector
from tensorflow.keras.layers import LSTM, Reshape
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, LeakyReLU
#from tensorflow.python.keras.engine.saving import model_from_json
from tensorflow.python.training.adam import AdamOptimizer
from tensorflow.keras import Input
#from tensorflow_core.python.keras.layers import concatenate, Concatenate, Bidirectional
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.models import load_model
#import matplotlib #linux
#matplotlib.use('Agg') #linux
import matplotlib.pyplot as plt


# Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.7.0 and strictly below 2.10.0 (nightly versions are not supported).
 # The versions of TensorFlow you are currently using is 2.2.0 and is not supported.
# import tensorflow_addons as tfa
# import tqdm
# # quietly deep-reload tqdm
# import sys
# from IPython.lib import deepreload
# stdout = sys.stdout
# sys.stdout = open('junk','w')
# deepreload.reload(tqdm)
# sys.stdout = stdout
# tqdm_callback = tfa.callbacks.TQDMProgressBar()

# import sys
# sys.stdout = open("test.txt", "w")

#class HLSTM:
#with tf.device('/gpu:0'):

EMBEDDING_DIM = 100
#MAX_NB_WORDS = 100000
text = []
labels = []

#activation='LeakyRelu'
#activation='selu'
activation='selu'
#optimizer = 'Adadelta'
optimizer = 'Adam'
#learning_rate = 0.00001
#optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
dropout = 0.5
neurons = 16
patience = 10
epochs = 20
batch_size = 128
folds = 5

def save_model(model, filename):
    model_json = model.to_json()
    with open(filename + '.model', "w") as json_file:
        json_file.write(model_json)
        json_file.close();
    model.save_weights(filename + ".weights")


# def load_model(filename):
#     json_file = open(filename + '.model', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     # loaded_model = model_from_json(loaded_model_json)
#     # loaded_model.load_weights(filename + ".weights")
#     # return loaded_model;


def getEmbeddingLayer():
    embeddings_index = {}
    countt = 0
    words = []
    #f = open('data/word_embeddings/glove.840B.300d.txt', encoding='utf-8', errors='ignore')
    #f = open('data/word_embeddings/glove.6B.100d.txt', encoding='utf-8')
    f = open('/content/drive/MyDrive/Colab Notebooks/TestCode/glove.6B.100d.txt')
    #f = open('word_embeddings/glove.6B.100d.txt')

    for line in f:
        values = line.split()  #espacio para glove 840b only
        word = values[0]
        words.append(word)
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        countt = countt + 1;
    f.close()

    # fin = io.open('data/word_embeddings/GoogleNews-vectors-negative300.bin', 'r', encoding='utf-8', newline='\n',
    #               errors='ignore')
    # n, d = map(int, fin.readline().split())
    # for line in fin:
    #     tokens = line.rstrip().split(' ')
    #     word = tokens[0]
    #     words.append(word)
    #     embeddings_index[tokens[0]] = map(float, tokens[1:])
    #     count += 1
    # fin.close()

    #tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer = Tokenizer(num_words=len(words)+1)
    tokenizer.fit_on_texts(words)
    word_index = tokenizer.word_index
    vocab_size = len(word_index) + 1

    print("total words embeddings is ", countt, len(word_index))
    embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    for word, i in word_index.items():
        # if i > vocab_size - 1:
        #     break
        # else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(input_dim=vocab_size,
                                output_dim=EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)

    return tokenizer, embedding_layer


# seed = 7346
# np.random.seed(seed)
count = 0
precision_list = list()
recall_list = list()
accuracy_list = list()
f1_list = list()


def create_embedded_model(embedding_layer, num_classes, MAX_SEQUENCE_LENGTH):
    input = Input(shape=(MAX_SEQUENCE_LENGTH,))
    emb_layer = embedding_layer(input)
    #model.add(tf.keras.layers.Dense(32, kernel_initializer='lecun_normal', activation='selu'))
    out = Flatten()(emb_layer)
    #out = Dense(32, kernel_initializer='lecun_normal', activation='selu')(out)
    out = Dense(neurons, kernel_initializer='lecun_normal', activation=activation)(out)
    #out = Dense(neurons, kernel_initializer='lecun_normal')(out)
    #out = LeakyReLU()(out)
    out = Dropout(rate=dropout)(out)
    output = Dense(2, activation='sigmoid')(out)
    modell = Model(input, output)
    print(modell.summary())
    return modell;


# Start here

dataframe = pd.DataFrame()
#dataframe = pd.read_csv("./data/sgd single-domains/sgd_act_Flights.csv", encoding='ISO-8859-1', sep='^', engine='python')
#dataframe = pd.read_csv("./data/sgd single-domains/sgd_act_Alarm.csv", encoding='ISO-8859-1', sep='^', engine='python')
#dataframe = pd.read_csv("data/sgd single-domains/sgd-act-prec1-Flights.csv", encoding='ISO-8859-1', sep='^', engine='python')
dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sarcasm_v2(0-1).csv", encoding='ISO-8859-1', sep=',', engine='python')
#dataframe = pd.read_csv("sarcasm_v2(0-1).csv", encoding='ISO-8859-1', sep=',', engine='python')

print(dataframe.head())
# dataframe = dataframe.replace(to_replace=r'\s,,\s', value=' ')
print(dataframe.size)

# cols = ['dialogue','text','label']
# cols = ['dialogue', 'text', 'prec_utt', 'prec_intent', 'label']
cols = ['Corpus', 'Label', 'ID', 'Quote Text', 'Response Text']

dataframe.columns = cols
#D = dataframe.loc[:, 'dialogue']
X = dataframe.loc[:, 'Response Text']
# A = dataframe.loc[:, 'prec_utt']
B = dataframe.loc[:, 'Quote Text']
Y = dataframe.loc[:, 'Label']
print(dataframe.shape)
#X = X.reshape((len(X),1))

X_featured = []
for i in range(X.count()):
    X_featured.append(X[i] + ' ' + B[i])
	  #X_featured.append(X[i] + ' ' + A[i] + ' ' + B[i])
X_featured = np.reshape(X_featured, X.shape)
#print(X_featured.shape)
#print(dataframe.shape)
# X = X.reshape((len(X),1))
print(X.count())
print(Y.count())

MAX_SEQUENCE_LENGTH = max([len(s.split()) for s in X])
tokenizer, embedding = getEmbeddingLayer()

sequences_x = tokenizer.texts_to_sequences(X_featured) #X_featured
# sequences_a = tokenizer.texts_to_sequences(A)
# sequences_b = tokenizer.texts_to_sequences(B)
#sequences_a2 = tokenizer.texts_to_sequences(A2)
#sequences_b2 = tokenizer.texts_to_sequences(B2)

#dd = pad_sequences(sequences_d, maxlen=MAX_SEQUENCE_LENGTH)
xx = pad_sequences(sequences_x, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating="post")
# aa = pad_sequences(sequences_a, maxlen=MAX_SEQUENCE_LENGTH)
# bb = pad_sequences(sequences_b, maxlen=MAX_SEQUENCE_LENGTH)
#aa2 = pad_sequences(sequences_a2, maxlen=MAX_SEQUENCE_LENGTH)
#bb2 = pad_sequences(sequences_b2, maxlen=MAX_SEQUENCE_LENGTH)

# d = np.transpose(d)
data = xx
#print(data)
uniques, ids = np.unique(Y, return_inverse=True)
y_train = tf.keras.utils.to_categorical(ids, len(uniques))
# y_train = np.reshape(y_train, (Y.count(),len(uniques)))
# y_train = keras.utils.to_categorical(Y,Y.count())
#print(len(uniques))
#print(y_train)
# y_train = np.array(y_train.shape[1])
# df = pd.DataFrame(data=y_train.data)
# y_train = np.asarray(Y)



  Corpus  Label             ID  \
0    GEN      1  GEN_sarc_0000   
1    GEN      1  GEN_sarc_0001   
2    GEN      1  GEN_sarc_0002   
3    GEN      1  GEN_sarc_0003   
4    GEN      1  GEN_sarc_0004   

                                          Quote Text  \
0  First off, That's grade A USDA approved Libera...   
1  watch it. Now you're using my lines. Poet has ...   
2  Because it will encourage teens to engage in r...   
3  Obviously you missed the point. So sorry the t...   
4  This is pure paranoia. What evidence do you ha...   

                                       Response Text  
0  Therefore you accept that the Republican party...  
1  More chattering from the peanut gallery? Haven...  
2  Yep, suppressing natural behavior is always th...  
3  I guess we all missed your point Justine, what...  
4  Evidence, I dont need no sticking evidence. Th...  
23460
(4692, 5)
4692
4692
total words embeddings is  400000 339251


###DNN_Algorithm_Part2

In [ ]:
print("Training data")
print(data.shape, y_train.shape)

# np.random.seed(1)
# y = np.random.randint(0, len(num_classes)-1, y_train.shape)
# y = y[np.where(y.sum(axis=1) != 0)[0]]
# y_new = LabelEncoder().fit_transform([''.join(str(l)) for l in y])
# print(y_new)
fig, ax = plt.subplots(figsize=(14, 8))
historyl = list()
# cross validation
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=0)
#optimizer = AdamOptimizer(learning_rate=0.0001)
#optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
for train, test in kfold.split(data, Y):
    # x_train, x_test, Y_train, Y_test = train_test_split(data, y_train, test_size=0.8, random_state=0)
    count += 1
    print('Fold: {0}'.format(count))
    print(data[train].shape, y_train[train].shape)
    model = create_embedded_model(embedding, len(uniques), MAX_SEQUENCE_LENGTH)

    # adad = tf.train.AdadeltaOptimizer()
    # adam = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    print('Training...')
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
    mc = ModelCheckpoint('best_model'+str(count)+'.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)
    # model.fit(np.reshape(data[train],(-1,data[train].shape[0],data[train].shape[1])), y_train[train], epochs=50, batch_size=batch_size, verbose=1, steps_per_epoch=20)  # 50 , steps_per_epoch=20
    history = model.fit(data[train], y_train[train], validation_split=0.1, epochs=epochs, batch_size=batch_size, verbose=2, callbacks=[es, mc])  # 50 , steps_per_epoch=20
    historyl.append(history)
    # save_model(model, "data/model")
    saved_model = load_model('best_model'+str(count)+'.h5')
    # HASTA AQUÃ  OK------------------------------------------------------------

    # TESTING
    print()
    print("Testing data")
    print(data[test].shape, y_train[test].shape)

    score, acc = saved_model.evaluate(data[test], y_train[test], verbose=1)
    print("acc: ", acc)
    print("Accuracy: {0:.2%}".format(acc))
    print('Test score:', score)
    y_predict = saved_model.predict(data[test])
    # y_predict = np.argmax(y_predict, axis=1)
    print(y_predict.shape)
    labels_pred = np.argmax(np.round(y_predict),axis=1)
    labels_test = np.argmax(y_train[test], axis=1)
    print(labels_test)
    print(labels_pred)
    #print(y_train[test], np.round(y_predict))
    accur = accuracy_score(y_train[test], np.round(y_predict))
    # print(roc_auc_score(Y[test], y_predict, average='macro'))
    print('Test accuracy:', accur)
    #print(confusion_matrix(np.argmax(y_train[test],axis=1), np.round(y_predict).argmax(axis=1)))
    b_acc = balanced_accuracy_score(labels_test, labels_pred)
    recall = recall_score(labels_test, labels_pred, average='weighted')
    precision = precision_score(labels_test, labels_pred, average='weighted', labels=np.unique(labels_pred))
    f1 = f1_score(labels_test, labels_pred, average='weighted')
    print('Test b_acc:', b_acc)
    print('Test precision:', precision)
    print('Test recall:', recall)
    print('Test f1:', f1)
    accuracy_list.append(b_acc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)
    print(classification_report(labels_test, labels_pred))
    # print(history.history.keys())
    plt.plot(history.history['loss'], color='blue')
    plt.plot(history.history['val_loss'], color='red')
    # plt.title(str(count))
    #plt.show()
    # fig.savefig(str(count)+'.png', bbox_inches='tight', pad_inches=0)

plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','validation'], loc='upper left')
# plt.show()
# plt.boxplot(accuracy_list)
plt.show()
# fig.savefig('box.png', bbox_inches='tight', pad_inches=0)
print('avg b_accuracy precision recall f1')
# print('avg b_accuracy {0}'.format(np.average(accuracy_list)))
# print('avg precision {0}'.format(np.average(precision_list)))
# print('avg recall {0}'.format(np.average(recall_list)))
# print('avg f1 {0}'.format(np.average(f1_list)))
print(np.average(accuracy_list))
print(np.average(precision_list))
print(np.average(recall_list))
f1_final = np.average(f1_list)
print(f1_final)

fig.savefig('loss_dnn(patience='+str(patience)+', epochs='+str(epochs)+', batch_size='+str(batch_size)+', neurons= '+str(neurons)+', activation='+str(activation)+', Dropout='+str(dropout)+', fold='+str(folds)+', optimizer='+str(optimizer)+', f1=' + str(f1_final) +').png', bbox_inches='tight', pad_inches=0)

#plt.figure()
# sys.stdout.close()